In [2]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.1 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125924 sha256=1038e4cfc6b8bf4bd4eb39b41d97ad61d44daaa0a08d584fb4bc7bbf9b347d27
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/MyDrive/en2sparql

Mounted at /content/drive
/content/drive/MyDrive/en2sparql


In [14]:
import json
import torch
from sentence_transformers import util


def save_json(filename, data):
    with open(filename, "w", encoding="utf-8") as json_file:
        print(json.dumps(data), file=json_file)


def main():
    corpus_embeddings = torch.load("train_embeddings.pt", map_location=torch.device('cpu'))
    questions = corpus_embeddings.get("questions")
    keys = corpus_embeddings.get("keys")
    corpus = []
    for i, question in enumerate(questions):
        corpus.append(keys[i]+ " - " + question)
    print(len(corpus))

    embeddings = (corpus_embeddings.get("emb_questions"))
    print(len(embeddings))

    t = torch.from_numpy(embeddings)
    clusters = util.community_detection(t, min_community_size=5, threshold=0.9)

    for i, cluster in enumerate(clusters):
        print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
        for sentence_id in cluster[0:3]:
            print("\t", corpus[sentence_id])
        print("\t", "...")
        for sentence_id in cluster[-3:]:
            print("\t", corpus[sentence_id])

    json_clusters = {}
    for i, cluster in enumerate(clusters):
        json_clusters["Cluster " + str(i + 1)] = []
        for sentence_id in cluster:
            json_clusters["Cluster " + str(i + 1)].append(corpus[sentence_id])

    save_json("sem_clusters.json", json_clusters)


main()

1795
1795

Cluster 1, #27 Elements 
	 5-T02 - List the title and ID of research papers that contain a benchmark over the Atari 2600 Battle Zone dataset?
	 5-T02 - List the title and ID of research papers that contain a benchmark over the Atari 2600 Alien dataset?
	 5-T02 - Give me a list of research papers along with their titles and IDs, that have performed benchmarks on the Atari 2600 Kangaroo dataset?
	 ...
	 5-T02 - Give me a list of research papers along with their titles and IDs, that have performed benchmarks on the Atari 2600 Ms. Pacman dataset?
	 5-T02 - Give me a list of research papers along with their titles and IDs, that have performed benchmarks on the Atari 2600 Freeway dataset?
	 5-T02 - Provide a list of research paper titles and IDs that have benchmarked models on the Atari 2600 Fishing Derby dataset?

Cluster 2, #15 Elements 
	 12-T05 - Indicate the model that performed best in terms of Score metric on the Atari 2600 Double Dunk benchmark dataset?
	 12-T05 - Indicate